# json

### 한페이지 수집

In [1]:
import requests
import os
import re
import time
import pandas as pd
from dotenv import load_dotenv
load_dotenv(dotenv_path="./data/.env")

True

In [2]:
def text_clean(text):
    # HTML 태그를 없애는 정규표현식
    result = re.sub(r"</?[^>]+>", "", text)
    # 한글, 영문, 숫자 외의 모든 문자 제거 후 공백으로 변환
    result = re.sub(r"[^가-힣a-zA-Z0-9]", " ", result)
    result = result.replace("  ", " ").replace("  ", " ").replace("  ", " ")
    return result

In [20]:
api_key = os.getenv("seoul_api")
url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/VwsmAdstrdNcmCnsmpW/1/5/"
r = requests.get(url)
#print(r.url)
#print(r.status_code)
response = r.json()
response

rows = response["VwsmAdstrdNcmCnsmpW"]["row"]

result={}
for item in rows:
    for key,value in item.items():
        result.setdefault(key, []).append(value)
        
df = pd.DataFrame(result)
df

,STDR_YYQU_CD,ADSTRD_CD,ADSTRD_CD_NM,MT_AVRG_INCOME_AMT,INCOME_SCTN_CD,EXPNDTR_TOTAMT,FDSTFFS_EXPNDTR_TOTAMT,CLTHS_FTWR_EXPNDTR_TOTAMT,LVSPL_EXPNDTR_TOTAMT,MCP_EXPNDTR_TOTAMT,TRNSPORT_EXPNDTR_TOTAMT,EDC_EXPNDTR_TOTAMT,PLESR_EXPNDTR_TOTAMT,LSR_CLTUR_EXPNDTR_TOTAMT,ETC_EXPNDTR_TOTAMT,FD_EXPNDTR_TOTAMT
0,20241,11545690,시흥3동,2516251.0,06,5.015260e+08,1.231490e+08,1.842000e+06,25596000.0,1.731500e+07,1.894620e+08,7.185000e+06,1.964800e+07,1.438000e+07,9.216000e+06,9.373300e+07
1,20191,11110530,사직동,5208463.0,09,3.188934e+10,1.238314e+09,1.589740e+08,60281000.0,5.864270e+08,1.884580e+08,1.692340e+08,2.937970e+08,1.241791e+09,2.367044e+10,4.281627e+09
2,20191,11110615,종로1·2·3·4가동,3350350.0,07,7.758461e+10,3.618998e+09,7.478066e+09,861970000.0,4.123252e+09,3.236609e+09,2.062644e+09,2.398515e+09,1.352075e+10,2.372902e+10,1.655478e+10
3,20191,11110650,혜화동,3472006.0,07,7.826497e+09,1.354887e+09,2.156650e+08,77222000.0,1.129580e+09,2.014500e+07,1.952790e+08,4.151960e+08,5.659230e+08,3.619260e+08,3.490674e+09
4,20191,11110680,창신2동,2179485.0,05,6.709870e+08,1.431950e+08,2.332300e+07,11149000.0,1.427540e+08,9.450000e+05,0.000000e+00,1.741100e+07,3.251300e+07,1.810400e+07,2.815930e+08


In [14]:
total_page = response["VwsmAdstrdNcmCnsmpW"]['list_total_count'] // 1000 + 1
total_page

12

### 전체 페이지 수집

In [21]:
import requests
import os
import re
import time
import pandas as pd
from dotenv import load_dotenv
load_dotenv(dotenv_path="./data/.env")

True

In [23]:
page_num = 1
page_size = 1000
total_page = 1
result = {}

url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/VwsmAdstrdNcmCnsmpW/1/{page_size}/"
r = requests.get(url)
response = r.json()
total_count = response["VwsmAdstrdNcmCnsmpW"]['list_total_count']
total_page = total_count // page_size + (1 if total_count % page_size != 0 else 0)

while page_num <= total_page:
    start = (page_num - 1) * page_size + 1
    end = total_count if page_num == total_page else page_num * page_size
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/VwsmAdstrdNcmCnsmpW/{start}/{end}/"
    
    r = requests.get(url)
    data = r.json()
    rows = data["VwsmAdstrdNcmCnsmpW"]["row"]
    
    for item in rows:
        for key,value in item.items():
            result.setdefault(key, []).append(value)
            
    print(f"{page_num}/{total_page}데이터 수집 중", end="\r")
    page_num += 1
    time.sleep(0.2)
        
df = pd.DataFrame(result)
df

,STDR_YYQU_CD,ADSTRD_CD,ADSTRD_CD_NM,MT_AVRG_INCOME_AMT,INCOME_SCTN_CD,EXPNDTR_TOTAMT,FDSTFFS_EXPNDTR_TOTAMT,CLTHS_FTWR_EXPNDTR_TOTAMT,LVSPL_EXPNDTR_TOTAMT,MCP_EXPNDTR_TOTAMT,TRNSPORT_EXPNDTR_TOTAMT,EDC_EXPNDTR_TOTAMT,PLESR_EXPNDTR_TOTAMT,LSR_CLTUR_EXPNDTR_TOTAMT,ETC_EXPNDTR_TOTAMT,FD_EXPNDTR_TOTAMT
0,20241,11545690,시흥3동,2516251.0,06,5.015260e+08,1.231490e+08,1.842000e+06,25596000.0,1.731500e+07,1.894620e+08,7.185000e+06,1.964800e+07,1.438000e+07,9.216000e+06,9.373300e+07
1,20191,11110530,사직동,5208463.0,09,3.188934e+10,1.238314e+09,1.589740e+08,60281000.0,5.864270e+08,1.884580e+08,1.692340e+08,2.937970e+08,1.241791e+09,2.367044e+10,4.281627e+09
2,20191,11110615,종로1·2·3·4가동,3350350.0,07,7.758461e+10,3.618998e+09,7.478066e+09,861970000.0,4.123252e+09,3.236609e+09,2.062644e+09,2.398515e+09,1.352075e+10,2.372902e+10,1.655478e+10
3,20191,11110650,혜화동,3472006.0,07,7.826497e+09,1.354887e+09,2.156650e+08,77222000.0,1.129580e+09,2.014500e+07,1.952790e+08,4.151960e+08,5.659230e+08,3.619260e+08,3.490674e+09
4,20191,11110680,창신2동,2179485.0,05,6.709870e+08,1.431950e+08,2.332300e+07,11149000.0,1.427540e+08,9.450000e+05,0.000000e+00,1.741100e+07,3.251300e+07,1.810400e+07,2.815930e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11045,20243,11620735,대학동,2622408.0,06,2.565461e+09,1.082769e+09,1.444100e+07,3555000.0,3.063140e+08,1.830000e+06,7.313600e+07,8.393800e+07,1.253240e+08,8.051700e+07,7.936370e+08
11046,20243,11650570,반포2동,6812331.0,09,5.913900e+08,1.703870e+08,1.136000e+06,0.0,8.921500e+07,3.507000e+06,1.517760e+08,0.000000e+00,2.447600e+07,1.883600e+07,1.320570e+08
11047,20243,11650590,방배본동,4856162.0,08,6.368143e+09,3.377150e+08,3.851900e+07,50812000.0,3.435310e+08,1.988340e+08,1.299940e+08,2.133200e+07,1.357380e+08,4.685978e+09,4.256900e+08
11048,20243,11680640,역삼1동,3607265.0,07,4.628527e+11,2.875360e+09,1.065433e+09,347696000.0,1.085545e+10,2.171190e+08,2.313910e+09,1.022743e+09,3.762154e+11,6.075319e+10,7.186400e+09


In [29]:
df.to_csv("./data/서울시상권분석서비스.csv", index=False,encoding="utf-8-sig")

# xml

### 한페이지 수집

In [31]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import re
from datetime import datetime
load_dotenv(dotenv_path="./data/.env")

True

In [33]:
api_key = os.getenv("seoul_api")
url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/VwsmAdstrdNcmCnsmpW/1/5/"
r = requests.get(url)
print(r.url)
print(r.status_code)
response = r.text     #xml은 response = r.text
response

http://openapi.seoul.go.kr:8088/45776d625173656f3838744b6f6c65/xml/VwsmAdstrdNcmCnsmpW/1/5/
200


'<?xml version="1.0" encoding="UTF-8"?>\n<VwsmAdstrdNcmCnsmpW>\n<list_total_count>11050</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>정상 처리되었습니다</MESSAGE>\n</RESULT>\n<row>\n<STDR_YYQU_CD>20241</STDR_YYQU_CD>\n<ADSTRD_CD>11545690</ADSTRD_CD>\n<ADSTRD_CD_NM>시흥3동</ADSTRD_CD_NM>\n<MT_AVRG_INCOME_AMT>2516251</MT_AVRG_INCOME_AMT>\n<INCOME_SCTN_CD>06</INCOME_SCTN_CD>\n<EXPNDTR_TOTAMT>501526000</EXPNDTR_TOTAMT>\n<FDSTFFS_EXPNDTR_TOTAMT>123149000</FDSTFFS_EXPNDTR_TOTAMT>\n<CLTHS_FTWR_EXPNDTR_TOTAMT>1842000</CLTHS_FTWR_EXPNDTR_TOTAMT>\n<LVSPL_EXPNDTR_TOTAMT>25596000</LVSPL_EXPNDTR_TOTAMT>\n<MCP_EXPNDTR_TOTAMT>17315000</MCP_EXPNDTR_TOTAMT>\n<TRNSPORT_EXPNDTR_TOTAMT>189462000</TRNSPORT_EXPNDTR_TOTAMT>\n<EDC_EXPNDTR_TOTAMT>7185000</EDC_EXPNDTR_TOTAMT>\n<PLESR_EXPNDTR_TOTAMT>19648000</PLESR_EXPNDTR_TOTAMT>\n<LSR_CLTUR_EXPNDTR_TOTAMT>14380000</LSR_CLTUR_EXPNDTR_TOTAMT>\n<ETC_EXPNDTR_TOTAMT>9216000</ETC_EXPNDTR_TOTAMT>\n<FD_EXPNDTR_TOTAMT>93733000</FD_EXPNDTR_TOTAMT>\n</row>\n<row>\n<

In [34]:
print(type(response))

<class 'str'>


In [35]:
from bs4 import BeautifulSoup as bs

In [36]:
soup = bs(response, 'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<VwsmAdstrdNcmCnsmpW>
<list_total_count>11050</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<STDR_YYQU_CD>20241</STDR_YYQU_CD>
<ADSTRD_CD>11545690</ADSTRD_CD>
<ADSTRD_CD_NM>시흥3동</ADSTRD_CD_NM>
<MT_AVRG_INCOME_AMT>2516251</MT_AVRG_INCOME_AMT>
<INCOME_SCTN_CD>06</INCOME_SCTN_CD>
<EXPNDTR_TOTAMT>501526000</EXPNDTR_TOTAMT>
<FDSTFFS_EXPNDTR_TOTAMT>123149000</FDSTFFS_EXPNDTR_TOTAMT>
<CLTHS_FTWR_EXPNDTR_TOTAMT>1842000</CLTHS_FTWR_EXPNDTR_TOTAMT>
<LVSPL_EXPNDTR_TOTAMT>25596000</LVSPL_EXPNDTR_TOTAMT>
<MCP_EXPNDTR_TOTAMT>17315000</MCP_EXPNDTR_TOTAMT>
<TRNSPORT_EXPNDTR_TOTAMT>189462000</TRNSPORT_EXPNDTR_TOTAMT>
<EDC_EXPNDTR_TOTAMT>7185000</EDC_EXPNDTR_TOTAMT>
<PLESR_EXPNDTR_TOTAMT>19648000</PLESR_EXPNDTR_TOTAMT>
<LSR_CLTUR_EXPNDTR_TOTAMT>14380000</LSR_CLTUR_EXPNDTR_TOTAMT>
<ETC_EXPNDTR_TOTAMT>9216000</ETC_EXPNDTR_TOTAMT>
<FD_EXPNDTR_TOTAMT>93733000</FD_EXPNDTR_TOTAMT>
</row>
<row>
<STDR_YYQU_CD>20191</STDR_YY

In [37]:
soup.select_one("list_total_count")

<list_total_count>11050</list_total_count>

In [38]:
soup.select_one("list_total_count").name

'list_total_count'

In [39]:
soup.select_one("list_total_count").text

'11050'

In [40]:
soup.select_one("list_total_count").string

'11050'

In [41]:
soup.select('row')

[<row>
 <STDR_YYQU_CD>20241</STDR_YYQU_CD>
 <ADSTRD_CD>11545690</ADSTRD_CD>
 <ADSTRD_CD_NM>시흥3동</ADSTRD_CD_NM>
 <MT_AVRG_INCOME_AMT>2516251</MT_AVRG_INCOME_AMT>
 <INCOME_SCTN_CD>06</INCOME_SCTN_CD>
 <EXPNDTR_TOTAMT>501526000</EXPNDTR_TOTAMT>
 <FDSTFFS_EXPNDTR_TOTAMT>123149000</FDSTFFS_EXPNDTR_TOTAMT>
 <CLTHS_FTWR_EXPNDTR_TOTAMT>1842000</CLTHS_FTWR_EXPNDTR_TOTAMT>
 <LVSPL_EXPNDTR_TOTAMT>25596000</LVSPL_EXPNDTR_TOTAMT>
 <MCP_EXPNDTR_TOTAMT>17315000</MCP_EXPNDTR_TOTAMT>
 <TRNSPORT_EXPNDTR_TOTAMT>189462000</TRNSPORT_EXPNDTR_TOTAMT>
 <EDC_EXPNDTR_TOTAMT>7185000</EDC_EXPNDTR_TOTAMT>
 <PLESR_EXPNDTR_TOTAMT>19648000</PLESR_EXPNDTR_TOTAMT>
 <LSR_CLTUR_EXPNDTR_TOTAMT>14380000</LSR_CLTUR_EXPNDTR_TOTAMT>
 <ETC_EXPNDTR_TOTAMT>9216000</ETC_EXPNDTR_TOTAMT>
 <FD_EXPNDTR_TOTAMT>93733000</FD_EXPNDTR_TOTAMT>
 </row>,
 <row>
 <STDR_YYQU_CD>20191</STDR_YYQU_CD>
 <ADSTRD_CD>11110530</ADSTRD_CD>
 <ADSTRD_CD_NM>사직동</ADSTRD_CD_NM>
 <MT_AVRG_INCOME_AMT>5208463</MT_AVRG_INCOME_AMT>
 <INCOME_SCTN_CD>09</INCOME_SC

In [42]:
markets = soup.select('row')

In [43]:
for market in markets:
    print(market.select_one('ADSTRD_CD_NM').text)

시흥3동
사직동
종로1·2·3·4가동
혜화동
창신2동


In [45]:
result = {}
for market in rows:
    for key,value in market.items():
         result.setdefault(key, []).append(value)
# dict함수의 key/value를 반복하려면 items() 사용해야함

df = pd.DataFrame(result)
df

,STDR_YYQU_CD,ADSTRD_CD,ADSTRD_CD_NM,MT_AVRG_INCOME_AMT,INCOME_SCTN_CD,EXPNDTR_TOTAMT,FDSTFFS_EXPNDTR_TOTAMT,CLTHS_FTWR_EXPNDTR_TOTAMT,LVSPL_EXPNDTR_TOTAMT,MCP_EXPNDTR_TOTAMT,TRNSPORT_EXPNDTR_TOTAMT,EDC_EXPNDTR_TOTAMT,PLESR_EXPNDTR_TOTAMT,LSR_CLTUR_EXPNDTR_TOTAMT,ETC_EXPNDTR_TOTAMT,FD_EXPNDTR_TOTAMT
0,20234,11710510,풍납1동,3011150.0,07,1.496979e+09,4.920910e+08,1.231000e+07,1.608550e+08,7.192000e+07,2.390500e+07,2.841200e+07,4.797600e+07,6.766400e+07,2.688090e+08,3.230370e+08
1,20234,11710600,석촌동,2927911.0,06,1.115530e+10,2.759076e+09,2.084210e+08,2.792980e+08,1.855966e+09,1.034690e+09,4.764700e+08,3.702110e+08,5.945770e+08,5.717030e+08,3.004890e+09
2,20234,11710642,문정2동,4425648.0,08,2.459714e+10,3.150538e+09,5.448103e+09,1.964078e+09,5.116893e+09,1.381222e+09,2.538220e+08,3.905050e+08,1.092209e+09,9.787660e+08,4.821009e+09
3,20234,11740530,명일1동,3540203.0,07,7.406202e+09,1.907591e+09,1.246810e+08,6.255800e+07,1.607272e+09,7.997100e+07,7.975930e+08,1.770670e+08,2.811470e+08,3.023690e+08,2.065953e+09
4,20234,11740570,암사1동,2852157.0,06,6.284657e+09,2.926651e+09,1.485160e+08,4.362700e+07,7.757220e+08,3.990750e+08,1.846990e+08,1.769450e+08,1.519230e+08,1.728280e+08,1.304671e+09
5,20234,11740600,천호1동,2473111.0,06,3.014987e+09,8.224590e+08,1.138250e+08,6.092100e+07,4.491290e+08,4.781600e+07,2.291190e+08,4.130300e+07,3.470240e+08,1.550370e+08,7.483540e+08
6,20234,11740685,길동,2840603.0,06,1.371159e+10,3.437519e+09,1.778980e+08,5.080630e+08,3.898244e+09,1.743580e+08,3.561430e+08,7.002560e+08,8.653510e+08,3.694030e+08,3.224353e+09
7,20234,11740700,둔촌2동,3414406.0,07,4.659856e+09,7.994600e+08,6.918100e+07,2.531640e+08,5.687580e+08,8.896340e+08,1.518740e+08,4.554000e+07,3.518170e+08,3.170730e+08,1.213355e+09
8,20242,11140635,약수동,3894878.0,08,2.739573e+09,1.084724e+09,3.899600e+07,3.509000e+06,4.998160e+08,1.022300e+07,7.667000e+07,5.785500e+07,1.812970e+08,9.646400e+07,6.900190e+08
9,20243,11500604,가양2동,3065320.0,07,1.233270e+09,7.420660e+08,1.348300e+07,2.045900e+07,1.863100e+08,8.744800e+07,7.396000e+06,6.070000e+06,6.169000e+07,2.339600e+07,8.495200e+07


### 전체 데이터 수집

In [48]:
import requests
import os
import re
import time
from dotenv import load_dotenv
import pandas as pd
load_dotenv(dotenv_path="./data/.env")
from bs4 import BeautifulSoup as bs

In [58]:
page_num = 1
total_page = 1
page_size = 1000
result = {}

while page_num <= total_page:
    api_key = os.getenv("seoul_api")
    start = (page_num -1) * page_size + 1
    end = page_num * page_size
    
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/VwsmAdstrdNcmCnsmpW/1/{start}/{end}/"
    r = requests.get(url)
    #print(r.url)
    #print(r.status_code)
    response = r.text
    soup = bs(response, 'xml') #반복문 안에 놔야 매 페이지마다 가져올 수 있음
                               #페이지마다 soup 새로 만들기!
    
    # 총 데이터 수
    if page_num == 1:
        total_count = int(soup.select_one("list_total_count").text)
        total_page = total_count // page_size + (1 if total_count % page_size != 0 else 0)

    rows = soup.find_all("row")
    for row in rows:
        for tag in row.find_all():
             result.setdefault(tag.name, []).append(tag.text)
                
    print(f"{page_num}/{total_page}데이터 수집 중", end="\r") 
    page_num += 1
    time.sleep(0.2)

df = pd.DataFrame(result)
df

,STDR_YYQU_CD,ADSTRD_CD,ADSTRD_CD_NM,MT_AVRG_INCOME_AMT,INCOME_SCTN_CD,EXPNDTR_TOTAMT,FDSTFFS_EXPNDTR_TOTAMT,CLTHS_FTWR_EXPNDTR_TOTAMT,LVSPL_EXPNDTR_TOTAMT,MCP_EXPNDTR_TOTAMT,TRNSPORT_EXPNDTR_TOTAMT,EDC_EXPNDTR_TOTAMT,PLESR_EXPNDTR_TOTAMT,LSR_CLTUR_EXPNDTR_TOTAMT,ETC_EXPNDTR_TOTAMT,FD_EXPNDTR_TOTAMT
0,20241,11545690,시흥3동,2516251,06,501526000,123149000,1842000,25596000,17315000,189462000,7185000,19648000,14380000,9216000,93733000


In [59]:
api_key = os.getenv("seoul_api")

page_num = 1
page_size = 500  # XML API 한 번에 너무 크게 요청하지 않기
result = {}

# 1페이지 요청해서 총 데이터 수 확인
url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/VwsmAdstrdNcmCnsmpW/1/{page_size}/"
r = requests.get(url)
soup = bs(r.text, 'xml')

total_count = int(soup.select_one("list_total_count").text)
total_page = total_count // page_size + (1 if total_count % page_size != 0 else 0)

# 전체 페이지 반복 수집
while page_num <= total_page:
    start = (page_num - 1) * page_size + 1
    end = page_num * page_size
    url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/VwsmAdstrdNcmCnsmpW/{start}/{end}/"
    
    r = requests.get(url)
    soup = bs(r.text, 'xml')  # 페이지마다 새로 만들기
    
    rows = soup.find_all("row")
    for row in rows:
        for tag in row.find_all():
            result.setdefault(tag.name, []).append(tag.text)
    
    print(f"{page_num}/{total_page} 페이지 수집 완료, 이번 페이지 row 수: {len(rows)}", end="\r")
    page_num += 1
    time.sleep(0.2)  # API 과부하 방지

df = pd.DataFrame(result)
df

,STDR_YYQU_CD,ADSTRD_CD,ADSTRD_CD_NM,MT_AVRG_INCOME_AMT,INCOME_SCTN_CD,EXPNDTR_TOTAMT,FDSTFFS_EXPNDTR_TOTAMT,CLTHS_FTWR_EXPNDTR_TOTAMT,LVSPL_EXPNDTR_TOTAMT,MCP_EXPNDTR_TOTAMT,TRNSPORT_EXPNDTR_TOTAMT,EDC_EXPNDTR_TOTAMT,PLESR_EXPNDTR_TOTAMT,LSR_CLTUR_EXPNDTR_TOTAMT,ETC_EXPNDTR_TOTAMT,FD_EXPNDTR_TOTAMT
0,20241,11545690,시흥3동,2516251,06,501526000,123149000,1842000,25596000,17315000,189462000,7185000,19648000,14380000,9216000,93733000
1,20191,11110530,사직동,5208463,09,31889341000,1238314000,158974000,60281000,586427000,188458000,169234000,293797000,1241791000,23670438000,4281627000
2,20191,11110615,종로1·2·3·4가동,3350350,07,77584610000,3618998000,7478066000,861970000,4123252000,3236609000,2062644000,2398515000,13520753000,23729021000,16554782000
3,20191,11110650,혜화동,3472006,07,7826497000,1354887000,215665000,77222000,1129580000,20145000,195279000,415196000,565923000,361926000,3490674000
4,20191,11110680,창신2동,2179485,05,670987000,143195000,23323000,11149000,142754000,945000,0,17411000,32513000,18104000,281593000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11045,20243,11620735,대학동,2622408,06,2565461000,1082769000,14441000,3555000,306314000,1830000,73136000,83938000,125324000,80517000,793637000
11046,20243,11650570,반포2동,6812331,09,591390000,170387000,1136000,0,89215000,3507000,151776000,0,24476000,18836000,132057000
11047,20243,11650590,방배본동,4856162,08,6368143000,337715000,38519000,50812000,343531000,198834000,129994000,21332000,135738000,4685978000,425690000
11048,20243,11680640,역삼1동,3607265,07,462852749000,2875360000,1065433000,347696000,10855447000,217119000,2313910000,1022743000,376215448000,60753193000,7186400000


In [61]:
df.to_csv("./data/서울시상권분석서비스.xml.csv", index=False, encoding="utf-8-sig")